In [1]:
!pip install -q transformers datasets accelerate bitsandbytes

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/DSC261 Final Project/Real Data/training_data.csv", on_bad_lines='skip', engine='python')

In [4]:
data

,id,article,highlights,article_length,cleaner_article,length_after_clean
0,363b7882753887539abef40e5e6f35a9df3ed572,"By . Leon Watson . PUBLISHED: . 10:31 EST, 17 ...","Pamela Hill, 51, visited grieving Susan Drisco...",3930,A grieving mother was left distraught when her...,3825
1,30538fe55f92f03a0a20bab02b8016f94241f376,(CNN) -- We know the point of the 2010 Census ...,"Chang-rae Lee: If census form were a person, h...",3251,We know the point of the 2010 Census is to cou...,3242
2,3c1ef6cfde146e104cebd6d8759fc8ebc015fe5f,Rapist footballer Ched Evans's girlfriend has ...,Rapist Ched Evans's girlfriend labelled a 'gol...,3221,Rapist footballer Ched Evans's girlfriend has ...,3221
3,4a606c3f0d69ffc317150e04e213b092bdeb5a7d,(CNN) -- Western companies operating in Africa...,Frank Braeken is the executive vice president ...,4626,Western companies operating in Africa have lon...,4617
4,069c2052ea4c5da5d2cc6255b0459be7565c87a8,By . Daily Mail Reporter . PUBLISHED: . 11:17 ...,"Liana Andrews, 16, and Kacie Watson, 14, went ...",2968,"Two teens in Cypress, Texas believed to have b...",2857
...,...,...,...,...,...,...
99995,65cd3efb93458bf75b2db40298c1676964c33455,"An autopsy is expected to be carried out, but ...","Margaret Holt, 65, left hotel in Costa Teguise...",1836,"An autopsy is expected to be carried out, but ...",1836
99996,0545d6613887c5dfce01ce2ea63b513d6af175ba,"By . Kerry Mcdermott . PUBLISHED: . 04:06 EST,...",Talae Akei Thomas said her newborn had died in...,2616,"Talae Thomas, 30, is understood to have confes...",2505
99997,ca274037ebb5be5990be02777075ca675d22769c,Hot diggity dog! The former home of Oscar G. M...,"The $1.75 million Evanston, Illinois, home fea...",2329,Hot diggity dog! The former home of Oscar G. M...,2329
99998,81db72d83bf667e6872ec8cbfcb5486d2b5892ab,(CNN) -- Erupting Icelandic volcanoes have bee...,Icelandair pilot loops back to give passengers...,1654,Erupting Icelandic volcanoes have been known t...,1645


In [5]:
train_data = data[['cleaner_article']]

In [6]:
from datasets import Dataset
dataset = Dataset.from_pandas(train_data, preserve_index=False)

In [7]:
def keep_article(example):
    return {"text": example["cleaner_article"].strip()}

dataset = dataset.map(keep_article, remove_columns=dataset.column_names)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [9]:
def tokenize(example):
    result = tokenizer(example["text"], truncation=True, max_length=1024, padding="max_length")
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split["train"]
eval_dataset = split["test"]

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = tokenizer.pad_token_id

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [17]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DSC261 Final Project/opt125m-news",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    lr_scheduler_type="cosine",
    learning_rate=1e-4,
    warmup_steps=100,
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    logging_steps=100,
    eval_steps=500,
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [ ]:
trainer.save_model("/content/drive/MyDrive/DSC261 Final Project/opt125m-news-model-base")
tokenizer.save_pretrained("/content/drive/MyDrive/DSC261 Final Project/opt125m-news-model-base")

Inference Time

In [3]:
# use T4?
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

save_dir = "/content/drive/MyDrive/DSC261 Final Project/opt125m-news-model-base"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    save_dir,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_dir)


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/DSC261 Final Project/opt125m-news-model-base'. Use `repo_type` argument if needed.

In [ ]:
synthetic = pd.read_csv("/content/drive/MyDrive/DSC261 Final Project/Real Data/training_data.csv", on_bad_lines='skip', engine='python')

In [ ]:
# split into 14k each

In [ ]:
# randomize the amount of words taken from the human article to generate something new
# randomize the amount of new tokens, between 500 - 1000
# randomize the temperature (decide what range looks best)

In [ ]:
def generate_text(input_text, max_new_tokens=700, temperature=0.5):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

  output = model.generate(
      input_ids,
      max_new_tokens=max_new_tokens,
      do_sample=True,
      temperature=temperature
  )

  return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
generate_text("The government has announced a new policy on renewable energy", 750, 0.8)